In [ ]:
import pandas as pd
import json
from datetime import datetime

In [ ]:
def flatten_json(nested_json):
    """
        Flatten json object with nested keys into a single level.
        Args:
            nested_json: A nested json object.
        Returns:
            The flattened json object if successful, None otherwise.
    """
#
# from https://towardsdatascience.com/how-to-flatten-deeply-nested-json-objects-in-non-recursive-elegant-python-55f96533103d
#
    out = {}

    def flatten(x, name=''):
        if type(x) is dict:
            for a in x:
                flatten(x[a], name + a + '_')
        elif type(x) is list:
            i = 0
            for a in x:
                flatten(a, name + str(i) + '_')
                i += 1
        else:
            out[name[:-1]] = x

    flatten(nested_json)
    return out

In [ ]:
#
# read and convert data
#
convert_line = lambda x: pd.Series(flatten_json(json.loads(x))).to_frame().T
e = None # flag for error and eof
file = open('recipeitems-latest_drop_records.json', 'r')
#
# the raw data were found to have variable columns in the records
# this allocates a fixed set of columns to speed things up
#
recipes = pd.DataFrame(columns = 
                       ['_id_$oid', 'name', 'ingredients', 'url', 'image', 'ts_$date',
                        'datePublished', 'source', 'recipeYield', 'prepTime', 'description',
                        'cookTime', 'totalTime', 'creator', 'recipeCategory', 'dateModified',
                        'recipeInstructions_0'])
temp_recipes = []
my_time = datetime.now()
lines = -1
while e == None:
    lines += 1
    if lines % 5000 == 0:
        print('loaded ', lines, ' lines')
    try:
        line = file.readline()
        if line == '':
            e = 'stop'
        else:
#
# this ensures each record is the same width, so we can use list append in the loop
#
            temp =  recipes.append(convert_line(line), ignore_index = True)
            temp_recipes.append(temp)
    except ValueError as x:
        e = None
file.close()
recipes = pd.concat(temp_recipes, axis = 0)
recipes.reset_index(drop = True, inplace = True)
recipes['ingredients'] = [recipes.loc[i, 'ingredients'].split('\n') for i in recipes.index]